In [1]:
import gc
import os
import talib
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timezone, time


import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from os import getenv
from dotenv import load_dotenv

from strategy import long_short, create_signal
from config import ccxt_connect, prices_collection, binance_client
from helper import vwap, current_position

load_dotenv()

client = ccxt_connect()
binance = binance_client()
prices = prices_collection()

In [11]:
df = pd.DataFrame(list(prices.find()))
df.sort_values('date', inplace=True)

df.drop_duplicates(subset='date', keep='last', inplace=True)

df.drop(['_id'], axis=1, inplace=True)

In [12]:
df

,date,close,open,high,low,vol
0,2019-09-23 08:39:59.999,209.55,209.55,209.55,209.55,0.01000
1,2019-09-23 08:44:59.999,209.55,209.55,209.55,209.55,0.00000
2,2019-09-23 08:49:59.999,209.55,209.55,209.55,209.55,0.00000
3,2019-09-23 08:54:59.999,209.55,209.55,209.55,209.55,0.00000
4,2019-09-23 08:57:17.144,209.55,209.55,209.55,209.55,0.00000
...,...,...,...,...,...,...
343623,2022-12-29 03:34:59.999,1192.24,1192.82,1193.13,1191.78,8.93152
343624,2022-12-29 03:39:59.999,1192.94,1192.08,1192.94,1191.82,15.27841
343619,2022-12-29 03:44:59.999,1193.40,1192.95,1193.40,1192.68,8.88215
343626,2022-12-29 03:49:59.999,1193.05,1193.26,1193.37,1192.98,13.17725


In [13]:
prices_collection().insert_many(df.to_dict('records'))

In [20]:
df = pd.DataFrame(list(prices.find().sort('date', -1)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
df['BB_up'], df['BB_mid'], df['BB_low'] = talib.BBANDS(close, timeperiod=20, nbdevup=2.3, nbdevdn=2.3, matype=0)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['atr'] = talib.ATR(df['high'].values, df['low'].values, close, timeperiod=14)
df['obv'] = talib.OBV(close, df['vol'].values)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

df.drop_duplicates(subset='date', keep='last', inplace=True)

print('Calculating signals')
df['cross'] = df.apply(long_short, axis=1)


df['Prev_High'] = df['high'].shift(1)
df['Prev_Low'] = df['low'].shift(1)
df['Prev_Close'] = df['close'].shift(1)

df['OO'] = df['open']-df['open'].shift(1)
df['OC'] = df['open']-df['Prev_Close']

df['Ret'] = (df['open'].shift(-1)-df['open'])/df['open']

t = .8
split = int(t*len(df))
split

# Create a column by name, 'Signal' and initialize with 0
df['signal'] = 0

# Assign a value of 1 to 'Signal' column for the quantile with the highest returns
df.loc[df['Ret'] > df['Ret'][:split].quantile(q=0.66), 'signal'] = 1

# Assign a value of -1 to 'Signal' column for the quantile with the lowest returns
df.loc[df['Ret'] < df['Ret'][:split].quantile(q=0.34), 'signal'] = -1

df = df[500:]
y = df['signal']
X = df.drop(['close', 'signal', 'high', 'low', 'vol', 'Ret', '_id'], axis=1)

Calculating signals


In [14]:
print('Starting the bot')
print(f"Current time: {datetime.utcnow().strftime('%m-%d-%y-%H:%M:%S')}")

last_date = prices.find().sort('date', -1).limit(1).next()['date']
print(f"Finding the last record in our DB: \n{last_date.strftime('%m-%d-%y-%H:%M:%S')}")

print(f'Querying data from binance to make current')
new_data_query = binance.get_klines(symbol='ETHUSDT', interval=binance.KLINE_INTERVAL_5MINUTE, endTime=int(last_date.timestamp() * 1000 + 1), limit=2)
print('Getting new data from Binance API (5m interval)')

new_data = pd.DataFrame(new_data_query[:-1], columns=['Open time', 'open', 'high', 'low', 'close', 'vol', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
new_data['date'] = pd.to_datetime(new_data['Close time'], unit='ms')
cols_to_drop = ['Open time', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)

if len(new_data) > 0:
    print(f'Inserting new data into DB')
    prices_collection().insert_many(new_data.to_dict('records'))

df = pd.DataFrame(list(prices.find().sort('date', -1).limit(300)))
df.sort_values('date', inplace=True)

df.drop_duplicates(subset='date', keep='last', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA15'] = talib.EMA(df.close, 15)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

print('Calculating signals')
df['cross'] = df.apply(long_short, axis=1)

df['signal'] = df[df['cross'] != 0]['cross'].diff()
df['signal'].fillna(0, inplace=True)

df['action'] = df.apply(create_signal, axis=1)


Starting the bot
Current time: 12-29-22-04:02:33
Finding the last record in our DB: 
12-29-22-03:54:59
Querying data from binance to make current
Getting new data from Binance API (5m interval)
Inserting new data into DB
Calculating signals


In [16]:
df.tail(15)

,_id,open,high,low,close,vol,rsi,macdhist,EMA15,EMA55,EMA200,date,cross,signal,action
285,63ad10f5e96509549fc4bcf3,1190.94,1192.06,1190.84,1191.59,8.29876,55.216294,0.012724,1190.233060,1190.573885,1192.644224,2022-12-29 02:49:59.999,0,0.0,None
286,63ad10f5e96509549fc4bcf4,1191.56,1191.99,1191.07,1191.46,10.31842,54.588794,0.109357,1190.386427,1190.605532,1192.632441,2022-12-29 02:54:59.999,0,0.0,None
287,63ad10f5e96509549fc4bcf5,1191.42,1191.52,1190.72,1191.52,5.21977,54.843862,0.170249,1190.528124,1190.638192,1192.621372,2022-12-29 02:59:59.999,0,0.0,None
288,63ad10f5e96509549fc4bcf6,1191.60,1191.84,1190.53,1191.03,19.97558,52.262141,0.170058,1190.590858,1190.652185,1192.605537,2022-12-29 03:04:59.999,0,0.0,None
289,63ad10f5e96509549fc4bcf7,1190.99,1191.44,1190.41,1190.41,14.68117,49.111870,0.122908,1190.568251,1190.643535,1192.583691,2022-12-29 03:09:59.999,0,0.0,None
290,63ad10f5e96509549fc4bcf8,1190.38,1190.39,1190.05,1190.15,1.58975,47.810356,0.071491,1190.515970,1190.625909,1192.559475,2022-12-29 03:14:59.999,0,0.0,None
291,63ad10f5e96509549fc4bcf9,1190.14,1192.00,1190.14,1191.72,15.24410,55.482313,0.136718,1190.666473,1190.664984,1192.551122,2022-12-29 03:19:59.999,2,1.0,Buy
292,63ad10f5e96509549fc4bcfa,1192.12,1192.33,1191.00,1192.12,28.73437,57.208257,0.196658,1190.848164,1190.716949,1192.546832,2022-12-29 03:24:59.999,2,0.0,None
293,63ad10f5e96509549fc4bcfb,1191.81,1192.86,1191.64,1192.33,9.86448,58.126128,0.237568,1191.033394,1190.774557,1192.544675,2022-12-29 03:29:59.999,2,0.0,None
294,63ad10f5e96509549fc4bcfc,1192.82,1193.13,1191.78,1192.24,8.93152,57.556328,0.244720,1191.184219,1190.826895,1192.541643,2022-12-29 03:34:59.999,2,0.0,None


In [73]:
binance.get_klines(symbol='ETHUSDT', interval=binance.KLINE_INTERVAL_5MINUTE, endTime=int(last_date.timestamp() * 1000 + 1000))

[[1672135800000,
  '1221.28000000',
  '1223.03000000',
  '1221.28000000',
  '1222.24000000',
  '13.48464000',
  1672136099999,
  '16479.04350770',
  30,
  '7.12947000',
  '8710.50695610',
  '0'],
 [1672136100000,
  '1222.64000000',
  '1222.64000000',
  '1221.75000000',
  '1222.24000000',
  '4.66651000',
  1672136399999,
  '5703.66885330',
  8,
  '3.65868000',
  '4472.03578710',
  '0'],
 [1672136400000,
  '1222.06000000',
  '1223.15000000',
  '1221.95000000',
  '1222.28000000',
  '5.90748000',
  1672136699999,
  '7221.67783100',
  11,
  '0.96634000',
  '1181.17209260',
  '0'],
 [1672136700000,
  '1222.38000000',
  '1222.82000000',
  '1222.38000000',
  '1222.61000000',
  '2.98981000',
  1672136999999,
  '3655.12190380',
  5,
  '2.08521000',
  '2549.14889780',
  '0'],
 [1672137000000,
  '1223.19000000',
  '1223.19000000',
  '1222.12000000',
  '1222.12000000',
  '15.68803000',
  1672137299999,
  '19181.00159110',
  21,
  '0.91520000',
  '1118.71302400',
  '0'],
 [1672137300000,
  '1222.160

In [49]:
df.tail(25)

,_id,open,high,low,close,vol,rsi,macdhist,EMA15,EMA55,EMA200,date,cross,signal,action
72,63ad039a990f26435a4c0f5e,1193.06,1193.22,1191.87,1192.05,10.73518,57.527477,0.263950,1191.469613,1191.001051,NaN,2022-12-29 01:09:59.999,2,0.0,None
73,63ad039a990f26435a4c0f5f,1192.16,1194.25,1192.16,1193.16,32.24726,62.056906,0.230287,1191.680912,1191.078156,NaN,2022-12-29 01:14:59.999,2,0.0,None
74,63ad053b02f50f8e43d391f1,1193.38,1193.38,1192.59,1192.59,8.80621,58.600887,0.152236,1191.794548,1191.132150,NaN,2022-12-29 01:19:59.999,2,0.0,None
75,63ad03f7990f26435a4c1154,1192.58,1193.19,1191.93,1192.47,21.21236,57.870198,0.078827,1191.878979,1191.179931,NaN,2022-12-29 01:24:59.999,2,0.0,None
76,63ad03f7990f26435a4c1155,1191.69,1192.21,1191.26,1191.88,7.29956,54.286161,-0.018000,1191.879107,1191.204933,NaN,2022-12-29 01:29:59.999,0,0.0,None
77,63ad05f0990f26435a4c134a,1192.26,1192.92,1192.05,1192.59,6.33917,57.682627,-0.041424,1191.967968,1191.254400,NaN,2022-12-29 01:34:59.999,0,0.0,None
78,63ad053b02f50f8e43d391f5,1192.57,1193.21,1191.76,1191.79,16.16116,52.912256,-0.113883,1191.945722,1191.273528,NaN,2022-12-29 01:39:59.999,0,0.0,None
79,63ad05f0990f26435a4c134c,1192.05,1193.50,1192.05,1193.08,6.82150,58.825429,-0.079639,1192.087507,1191.338045,NaN,2022-12-29 01:44:59.999,0,0.0,None
80,63ad039a990f26435a4c0f66,1192.97,1192.97,1191.78,1192.41,20.71555,54.964725,-0.105212,1192.127819,1191.376329,NaN,2022-12-29 01:49:59.999,0,0.0,None
81,63ad05f0990f26435a4c134e,1192.25,1192.25,1190.69,1190.90,11.53973,47.412407,-0.220304,1191.974341,1191.359318,NaN,2022-12-29 01:54:59.999,0,0.0,None


In [40]:
orders = client.fetch_closed_orders(symbol)

In [41]:
orders[-1]['info']['side']

'Sell'

In [42]:
int(orders[-1]['info']['qty'])

2681

In [44]:
signal = 'BUY'

In [45]:
if signal == 'BUY':
    print(f'Signal went from: {position} to {signal}, flipping to long at {price}')
    balance = client.fetch_balance()
    last_order = client.fetch_closed_orders(symbol)
    if last_order[-1]['info']['side'] == 'Sell':
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = int(last_order[-1]['info']['qty']),
                params = {'reduce_only': True})

    quantity = round((float(balance['ETH']['free']) * .95), 3)
    order = client.create_order(
            symbol = 'ETHUSD',
            type = 'market',
            side = 'buy',
            amount = (price * quantity) * 5,
            params = {'leverage': 5})
    print(order)

Signal went from: SELL to BUY, flipping to long at 1218.11
{'info': {'user_id': '48899600', 'order_id': '723dacdd-82ee-4bfb-8b9d-89428ea52bae', 'symbol': 'ETHUSD', 'side': 'Buy', 'order_type': 'Market', 'price': '1269.35', 'qty': '5536', 'time_in_force': 'ImmediateOrCancel', 'order_status': 'Created', 'last_exec_time': '0', 'last_exec_price': '0', 'leaves_qty': '5536', 'cum_exec_qty': '0', 'cum_exec_value': '0', 'cum_exec_fee': '0', 'reject_reason': 'EC_NoError', 'order_link_id': '', 'created_at': '2022-12-27T14:45:44.639Z', 'updated_at': '2022-12-27T14:45:44.639Z', 'take_profit': '0.00', 'stop_loss': '0.00', 'tp_trigger_by': 'UNKNOWN', 'sl_trigger_by': 'UNKNOWN'}, 'id': '723dacdd-82ee-4bfb-8b9d-89428ea52bae', 'clientOrderId': None, 'timestamp': 1672152344639, 'datetime': '2022-12-27T14:45:44.639Z', 'lastTradeTimestamp': None, 'symbol': 'ETH/USD:ETH', 'type': 'market', 'timeInForce': 'IOC', 'postOnly': False, 'side': 'buy', 'price': 1269.35, 'triggerPrice': None, 'stopPrice': None, 'am

In [47]:
order['info']['updated_at'], order['info']['symbol']

'ETHUSD'

In [ ]:
if signal == position or signal == None:
        print('No change in position, sleep for 5min and rescan')

else:
        with open('current_position.txt', 'w') as f:
                f.write(str(signal))

if signal == 'BUY':
        print(f'Signal went from: {position} to {signal}, flipping to long at {price}')
        balance = client.fetch_balance()
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'sell',
                amount = (price * balance['ETH']['used']) * 5,
                params = {'reduce_only': True})
        quantity = round((float(balance['ETH']['free']) * .95), 3)
        order = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = (price * quantity) * 5,
                params = {'leverage': 5})
        print(order)

if signal == 'SELL':
        print(f'Signal went from: {position} to {signal}, flipping to short at {price}')
        balance = client.fetch_balance()
        close = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'buy',
                amount = (price * balance['ETH']['used']) * 5,
                params = {'reduce_only': True})
        quantity = round((float(balance['ETH']['free']) * .95), 3)
        order = client.create_order(
                symbol = 'ETHUSD',
                type = 'market',
                side = 'sell',
                amount = (price * quantity) * 5,
                params = {'leverage': 5})
        print(order)

In [ ]:
df.to_csv('check.csv', index=False)

In [ ]:
df.plot(y=['close', 'EMA20'], x='date', figsize=(20, 10))